## File IO

In [ ]:
fileName = 'chat_sehreen.txt'

with open(fileName, encoding="utf8") as f:
    lineList = f.readlines()

In [ ]:
print('Length:',len(lineList))
print(lineList[0]+lineList[len(lineList)-1])

In [ ]:
import re
# print(lineList[0].split(','))
date = []
time = []
name = []
msg = []
i = 0
for line in lineList:
#     print(line.split(',',1))
    try:
        d, c = line.split(',',1)
        t, c = c.split(' - ',1)
        n, m = c.split(': ',1)
        
        date.append(d)
        time.append(t)
        name.append(n.replace('Jawad Aziz Khan', 'Yousha'))
        msg.append(m.strip('\n'))
    except:
        i+=1
print(i)    

In [ ]:
import pandas as pd
df = pd.DataFrame(list(zip(date, time, name, msg)), columns=['Date', 'Time', 'Name', 'Msg'])

In [ ]:
df.to_csv('chat.csv',index=0)

In [ ]:
df

## Date Alignment and Frequency

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv('chat.csv')

In [ ]:
df

In [ ]:
# df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y')

In [ ]:
df_you = df.loc[df['Name'] == 'Yousha']
df_seh = df.loc[df['Name'] == 'Sehreen']

In [ ]:
freqy = df_you.groupby('Date')['Name'].value_counts().tolist()
freqs = df_seh.groupby('Date')['Name'].value_counts().tolist()

datey = []
dates = []
for date in df_you.index:
    if date not in datey:
        datey.append(date.to_pydatetime())
for date in df_seh.index:
    if date not in dates:
        dates.append(date.to_pydatetime())
                 
print(len(freqy), len(datey))
print(len(freqs), len(dates))

In [ ]:
ploty = [freqy,datey]
plots = [freqs,dates]

### By Year

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

fig = plt.figure(figsize=[16,10])
ax = fig.add_subplot(1,1,1)  
plt.scatter(plots[1], plots[0], marker='o', label="Sehreen Msgs", color="violet")
plt.scatter(ploty[1], ploty[0], marker='X', label="Yousha Msgs", color="dodgerblue")

plt.grid()
ax.xaxis.set_major_locator(mdates.MonthLocator(interval=4))
ax.xaxis.set_minor_locator(mdates.MonthLocator(interval=1))
ax.xaxis.set_major_formatter(mdates.DateFormatter('%b \'%y'))
ax.legend()
ax.set_ylabel('Total messages / day')
ax.set_xlabel('Time in Months')
ax.set_title('Whatsapp Message Frequency Scatterplot - Yousha & Sehreen')
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)
plt.savefig('images/msgfreq_years.png', dpi=100)

### By Days in a Week

In [ ]:
from datetime import datetime
weekdays = ['Sunday','Monday','Tuesday','Wednesday','Thursday','Friday','Saturday']
days = []
# avgdaydate = df.Date.unique()
for date in df.Date:
    day = datetime.strptime(date, '%d/%m/%Y').strftime("%A")
    days.append(day)

In [ ]:
daycounts = []
for day in weekdays:
    print(day+':',days.count(day))
    daycounts.append(days.count(day))
print(daycounts)

In [ ]:
import math

avgdatecounts = df.groupby('Date')['Date'].value_counts().tolist()
avgdates = df['Date'].unique().tolist()

for i,d in enumerate(avgdates):
    avgdates[i] = datetime.strptime(d, '%d/%m/%Y').strftime("%A")

avg = []
for i,d in enumerate(weekdays):
    a = math.floor(daycounts[i]/avgdates.count(d))
    avg.append(a)
    print(d,avgdates.count(d),daycounts[i],'  Average:',a)

In [ ]:
import numpy as np
y_pos = np.arange(len(weekdays))

fig = plt.figure(figsize=[10,8])
plt.bar(y_pos, daycounts, align='center', color=['greenyellow', 'steelblue', 'deepskyblue', 'turquoise', 'slategray', 'mediumspringgreen', 'steelblue'])
plt.xticks(y_pos, weekdays)
plt.ylabel('Total Messages by Day')
plt.title('Whatsapp Message Frequency by Weekdays')
axes = plt.gca()
axes.set_ylim(bottom=35000)
axes.spines['top'].set_visible(False)
axes.spines['right'].set_visible(False)
plt.savefig('images/msgfreq_days.png', dpi=150)
plt.show()

In [ ]:
import numpy as np
y_pos = np.arange(len(weekdays))

fig = plt.figure(figsize=[10,8])
plt.bar(y_pos, avg, align='center', color=['mediumspringgreen', 'steelblue', 'deepskyblue', 'turquoise', 'slategray', 'turquoise', 'deepskyblue'])
plt.xticks(y_pos, weekdays)
plt.ylabel('Average No. of Messages by Day')
plt.title('Whatsapp Message Average Frequency by Weekdays')
axes = plt.gca()
axes.set_ylim(bottom=250)
axes.spines['top'].set_visible(False)
axes.spines['right'].set_visible(False)
plt.savefig('images/msgfreq_daysavg.png', dpi=150)
plt.show()

### By Time

In [ ]:
from datetime import datetime, timedelta
from collections import Counter

dftimes = []
for row in df.iterrows():
    dftimes.append(datetime.strptime(row[1]['Time'].strip(), '%I:%M %p'))
    
def ceil_dt(dt, delta):
    return dt + (datetime.min - dt) % delta

for i,d in enumerate(dftimes):
    dftimes[i] = ceil_dt(d, timedelta(minutes=30))

timescount = Counter(dftimes).keys() # equals to list(set(words))
timesfreq = Counter(dftimes).values() # counts the elements' frequency

In [ ]:
tf = []
tc = []
for i in timesfreq:
    tf.append(i)
for i in timescount:
    tc.append(i)

In [ ]:
import datetime
import random
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

# make up some data
x = tc
y = tf

# plot
fig = plt.figure(figsize=[10,8])
new_x, new_y = zip(*sorted(zip(x, y)))
plt.plot(new_x,new_y,color='dodgerblue',linewidth=3)

plt.ylabel('Average No. of Messages by Time of Day')
plt.xlabel('Time Messages Were Sent')
plt.title('Whatsapp Message Average Frequency by Time')

ax = plt.gca()
ax.set_xlim([datetime.datetime(1900, 1, 1,1,0), datetime.datetime(1900, 1, 2,0,0)])
myFmt = mdates.DateFormatter('%I:%M %p')
ax.xaxis.set_major_formatter(myFmt)
ax.xaxis.set_minor_locator(mdates.HourLocator(interval=1))
plt.gcf().autofmt_xdate()
plt.grid()

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
plt.savefig('images/msgfreq_time.png', dpi=150)

plt.show()

### By Words

In [ ]:
df.Name.value_counts()

In [ ]:
df.Msg.value_counts().head(20)

In [ ]:
f = open('msg_y.txt','w',encoding='utf8')
for msg in df_you.Msg:
    f.write(str(msg)+'\n')
f.close()
f = open('msg_s.txt','w',encoding='utf8')
for msg in df_seh.Msg:
    f.write(str(msg)+'\n')
f.close()

In [ ]:
from emoji import UNICODE_EMOJI

def is_emoji(s):
    return s in UNICODE_EMOJI

In [ ]:
import collections
import pandas as pd
import matplotlib.pyplot as plt

file = open('msg_y.txt', encoding="utf8")
a= file.read()

stopwords = set(line.strip() for line in open('stopwords.txt'))
stopwords = stopwords.union(set(['<media','omitted>','it\'s','two','said']))
wordcount = {}
for word in a.lower().split():
    word = word.replace(".","")
    word = word.replace(",","")
    word = word.replace("\"","")
    word = word.replace("!","")
    if word not in stopwords:
        if word not in wordcount:
            wordcount[word] = 1
        else:
            wordcount[word] += 1

emocount = {}
for e in a:
    if is_emoji(e):
        if e not in emocount:
            emocount[e] = 1
        else:
            emocount[e] +=1

words_y = [[],[]]
emo_y = [[],[]]
word_counter = collections.Counter(wordcount)
for word, count in word_counter.most_common(20):
    print(word, ": ", count)
    words_y[0].append(word)
    words_y[1].append(count)
print()
emo_counter = collections.Counter(emocount)
for e, c in emo_counter.most_common(20):
    print(e, ": ", c)
    emo_y[0].append(e)
    emo_y[1].append(c)
    
file.close()

In [ ]:
file = open('msg_s.txt', encoding="utf8")
a= file.read()

stopwords = set(line.strip() for line in open('stopwords.txt'))
stopwords = stopwords.union(set(['<media','omitted>','it\'s','am','said']))
wordcount = {}
for word in a.lower().split():
    word = word.replace(".","")
    word = word.replace(",","")
    word = word.replace("\"","")
    word = word.replace("!","")
    if word not in stopwords:
        if word not in wordcount:
            wordcount[word] = 1
        else:
            wordcount[word] += 1

emocount = {}
for e in a:
    if is_emoji(e):
        if e not in emocount:
            emocount[e] = 1
        else:
            emocount[e] +=1

words_s = [[],[]]
emo_s = [[],[]]
word_counter = collections.Counter(wordcount)
for word, count in word_counter.most_common(20):
    print(word, ": ", count)
    words_s[0].append(word)
    words_s[1].append(count)
print()
emo_counter = collections.Counter(emocount)
for e, c in emo_counter.most_common(20):
    print(e, ": ", c)
    emo_s[0].append(e)
    emo_s[1].append(c)
    
file.close()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

plt.rcdefaults()
fig,(ax,ax2) = plt.subplots(1,2,figsize=(10,8))
y_pos = np.arange(20)


ax.barh(y_pos, words_y[1], align='center', color='dodgerblue')
ax.spines['bottom'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.yaxis.tick_right()
ax.set_yticks(y_pos)
ax.set_yticklabels(words_y[0])
ax.invert_xaxis()
ax.get_xaxis().set_visible(False)
ax.invert_yaxis() 
for index, value in enumerate(words_y[1]):
    ax.text(value+750, index, ' {:.0f}   '.format(value))
# ax.set_title('How fast do you want to go today?')

ax2.barh(y_pos, words_s[1], align='center', color='violet')
ax2.spines['top'].set_visible(False)
ax2.spines['left'].set_visible(False)
ax2.spines['bottom'].set_visible(False)
ax2.spines['right'].set_visible(False)
ax2.set_yticks(y_pos)
ax2.set_yticklabels(words_s[0])
ax2.invert_yaxis()
ax2.get_xaxis().set_visible(False)
# ax.set_xlabel('Performance')
for index, value in enumerate(words_s[1]):
    ax2.text(value, index, ' {:.0f}'.format(value))
# ax2.set_title('How fast do you want to go today?')
# plt.show()
fig.suptitle('Top 20 Most Used Words')
fig.tight_layout(pad=3)

plt.savefig('images/topwords.png', dpi=150)
plt.show()

In [ ]:

plt.rcdefaults()
fig, ax2 = plt.subplots(figsize=(10,8))
y_pos = np.arange(20)

ax2.barh(y_pos, emo_s[1], align='center', color='violet')
ax2.spines['top'].set_visible(False)
ax2.spines['left'].set_visible(False)
ax2.spines['bottom'].set_visible(False)
ax2.spines['right'].set_visible(False)
ax2.set_yticks(y_pos)
ax2.set_yticklabels('')
ax2.invert_yaxis()
ax2.get_xaxis().set_visible(False)
# ax.set_xlabel('Performance')
for index, value in enumerate(emo_s[1]):
    ax2.text(value, index, ' {:.0f}'.format(value))
# ax2.set_title('How fast do you want to go today?')
# plt.show()
fig.suptitle('Top 20 Most Used Emojis - Sehreen')
fig.tight_layout(pad=3)

plt.savefig('images/topemojis.png', dpi=150)
plt.show()